# YeastOnSucroseAgar(1D)_Coop-Cheat

This script use a PDE solver (scikit-fdiff) to resolve numerically the 1D diffusion equation.
For the sucrose degradation, we use an hook in the PDE solver to acces and modify the concentration while the simulation run.
There are no cell movement.

To run this script, you need several packages (skfdiff,numpy,holoviews,csv).
You can also use the Anaconda environment containing all the necessary packages:
https://anaconda.org/matthias.lebec/CellModeling/files

### Import packages

In [ ]:
import numpy as np
from skfdiff import Model, Simulation
from skfdiff import display_fields, enable_notebook #for display in realtime
from skfdiff import Container #to retrieve from disk
import holoviews as hv
enable_notebook()
import csv


### Define variables

In [ ]:
pixelvalue=0.0001 #0.1mm [m]
sizeX=0.045 #[m]
nbrpixelX=int(sizeX/pixelvalue)

sizeZ=0.00235 #[m]
toplayerthickness=0.00067 #[m] 0.67mm
ratiolayers=toplayerthickness/sizeZ

tol=1e1 #error tolerance for numerical method
deltat=2 #in sec ; time step for numerical resolution
tmax=60*60*85 #in sec ; total time to compute
containerpath="D:/Matthias_LE_BEC/Modelling/Python/" #path to save the simulation results
containerid="TestGitHub_1" #name for the simulation

In [ ]:
#Diffusion coefficient in water: at 30°C
kM=7.6e-10 #m²/s
kS=6.1e-10 #m²/s

KmE=0.026 #Km of Invertase
Kcat=4700 #Kcat of Invertase
umax=0.27 #/h ; maximal yeast growth rate
umax = umax/3600 #in sec
Ks=0.00012 #Monod constant for yeast
Kalpha=0.00001 #Arbitrary Hill coeff for Invertase production
Death = 0 #Death rate
Vmax1=(167e-6/60)*(15e-12) #Mich.Ment. coeff for glucose consumption
Vmax2=(104e-6/60)*(15e-12) #Mich.Ment. coeff for glucose consumption
Km1=0.0008 #Mich.Ment. coeff for glucose consumption
Km2=0.021 #Mich.Ment. coeff for glucose consumption

print("Stability evaluation: delta t=",deltat," should be < " , pixelvalue*pixelvalue/kM)

In [ ]:
M0=0.00000001 #[M] ; Initial monomers concentration(Glu+Fruct)  0.005% = 0.00027M
S0=0.02921 #[M] ; Initial sucrose concentration  1% = 0.029M
E0=1e-35 #[M] ; Initial enzyme concentration
alphacheat=1.5e-25 #Invertase production rate[mol/s/cell] this is typically 1e-24 in WT
alphacoop=1.8e-24 #Invertase production rate[mol/s/cell] this is typically 1e-24 in WT

S0=S0*(1-ratiolayers) #take into account that sucrose is not in the whole thickness

CellD=1e13 #initial number of cell per m^3
CellD0=CellD/1000 #initial number of cell per L
CellD0=CellD0*ratiolayers #take into account that cell are not in the whole thickness

In [ ]:
#Initialising the fields
x = np.linspace(0, pixelvalue*(nbrpixelX-1), nbrpixelX) #Initialize the size of the window
M = [ M0 for i in range(nbrpixelX) ] #Initialise the field M with M0
S = [ S0 for i in range(nbrpixelX) ] #Initialise the field S with S0
En = [ E0 for i in range(nbrpixelX) ] #Initialise the field E with E0
alpha = [ alphacheat for i in range(nbrpixelX) ] #Initialise the field alpha with alphacheat
d = np.zeros(x.size)#Initialise the field with 0
Q= np.zeros(x.size)#Initialise the field with 0
I= np.zeros(x.size)#Initialise the field with 0
tracker= np.zeros(x.size)#Initialise the field with 0
for i in range(nbrpixelX):
    d[i]=CellD0
    
#Create the location of the cooperators:
listcoop=[]
center=int((nbrpixelX-1)/2)
ratioCoop=0.25 # area ratio of cooperator
pixelDMDsize=0.35e-3 #in meter ; size of one DMD pixel when projected on the surface
Wavelength=16*pixelDMDsize #in meter ; the light pattern wavelength
wl=Wavelength/pixelvalue #in pixel ; the light pattern wavelength

LightIntensity=1 #between 0 and 1
alphacoop=alphacheat+(alphacoop-alphacheat)*LightIntensity #compute alphacoop based on the light intensity: linearly

#Setup the light pattern
#Fill the listcoop
linecount=0
for i in range(nbrpixelX): # this is basically the blue pattern
    j=i-linecount*wl #This shift i depending on the number of the pattern
    if (j>(1-ratioCoop)*wl)&(j<=wl): 
        listcoop+=[i]
    if (j>wl):
        linecount+=1

#Fill the illuminated pixel with the invertase production rate alphacoop
for i in listcoop: 
    alpha[i]=alphacoop


### Define PDE model

In [ ]:
model = Model(["kM * (dxxM) - Q + 2*I",
               "kS * (dxxS) - I",
               "alpha* (M/(Kalpha+M))*d",
              "umax * (M/(Ks+M)) * d - Death*d"],
              ["M(x)", "S(x)","En(x)","d(x)"], 
              parameters=["kM","kS","umax","Ks","Kalpha","Death","Q(x)","I(x)","alpha(x)"],
              boundary_conditions="noflux")

### Hook

In [ ]:
def degrad_hook(t, fields):
    fields["M"] = ("x"), np.where(fields.M <= 0, 0, fields.M) #This prevent M to go negative
    fields["S"] = ("x"), np.where(fields.S <= 0, 0, fields.S) #This prevent S to go negative
    fields["En"] = ("x"), np.where(fields.En <= 0, 0, fields.En) #This prevent En to go negative
    fields["d"] = ("x"), np.where(fields.d <= 0, 0, fields.d) #This prevent d to go negative
    fields["Q"] = ("x"), (Vmax1*fields.M/(Km1+fields.M)+Vmax2*fields.M/(Km2+fields.M))*fields.d #Computing the glucose consumption
    fields["I"] = ("x"), fields.En*Kcat*fields.S/(KmE+fields.S) #Computing the invertase activity
    return fields

### Initialize simulation

In [ ]:
initial_fields = model.Fields(x=x, M=M, S=S, d=d, kM=kM, kS=kS,umax=umax,Ks=Ks,Kalpha=Kalpha,Death=Death, Q=Q, I=I, En=En, alpha=alpha) #Initialise the field, k is diffusion coefficient m/s ?

###Here you have the tolerence setting, very important for the script computation time
simulation = Simulation(model, initial_fields,dt=deltat, tmax=tmax,hook=degrad_hook,scheme="Theta",theta=0.5,time_stepping=True,tol=tol,id=containerid)
container = simulation.attach_container(containerpath, force=True)

### If you want to plot in real time while simulation is running

In [ ]:
enable_notebook()
#display_fields(simulation)

### Run the simulation

In [ ]:
for t, fields in simulation:
    print("- time step:",t/3600,"hour, S mean:" ,float(fields["S"].mean()),", M mean:" ,float(fields["M"].mean()))
    print("Q mean:" ,float(fields["Q"].mean()),"I mean:" ,float(fields["I"].mean()))
    print("time since simulation start:",simulation.timer.total)
    #fig = pl.figure()
    #fields["C"].plot()

In [ ]:
simulation.timer.total #Print the total computation time

In [ ]:
container = Container.retrieve(containerpath+containerid) #Recover the container data stored in the disk

# Plot the result 

In [ ]:
from xarray import concat, open_dataset, open_mfdataset
from path import Path

In [ ]:
path=Path(containerpath+containerid)

### For very big data:

In [ ]:
# listdataset=[]
# previousframe=0

# i=0
# for filename in path.files("data*.nc"):
#     print(filename)

#     listdataset+=[open_dataset(filename).isel(t=[0])] #This take only the first timepoint of each data*.nc file

### For normal data:

In [ ]:
listdataset=[]
previousframe=0

i=0
for filename in path.files("data*.nc"):
    #print(filename)

    listdataset+=[open_dataset(filename)]

In [ ]:
alldata = concat(listdataset,dim="t").sortby("t")

In [ ]:
skipframe=1000# the number of time frame you want to skip for the ploting in addition of the first sampling
start=0
end=150000

In [ ]:
hv.output(max_frames=1001) #Tune this "protection"
data=alldata.isel(t=slice(start,end,skipframe))

### Ploting:

In [ ]:
skipx=10 # the number of x position you want to skip for the ploting

In [ ]:
dprofil = hv.Dataset(data.d).to(hv.Curve, ["x"])
dprofil

In [ ]:
dcurve = hv.Dataset(data.d.sel(x=data.x[range(0,len(data.x),skipx)])).to(hv.Curve, ["t"])
dcurve

In [ ]:
Mcurve = hv.Dataset(data.M.sel(x=data.x[range(0,len(data.x),skipx)] )).to(hv.Curve, ["t"])
Mcurve

In [ ]:
Sprofil = hv.Dataset(data.S).to(hv.Curve, ["x"])
Sprofil

In [ ]:
Mprofil = hv.Dataset(data.M).to(hv.Curve, ["x"])
Mprofil

### Save the result in holoview

In [ ]:
AllPlot_t=(Sprofil+Mprofil+dprofil).cols(3)
AllPlot_x=(Mcurve+dcurve).cols(2)
hv.renderer('bokeh').save(AllPlot_t, containerid+'AllPlot_t', fmt='scrubber')
hv.renderer('bokeh').save(AllPlot_x, containerid+'AllPlot_x', fmt='scrubber')

### Save the result in csv

In [ ]:
path = Path('C:/Users/Mathias/Documents/Python Scripts/SucroseGrowthYeast/'+containerid)
try:
    path.mkdir()
except:
    print("already existing folder")

In [ ]:
###d

variablename="d"
with open(path+"/"+containerid+"_"+variablename+".csv", mode='w',newline='') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"')
    listx=[]
    for x in data.x.values:
        listx+=[x]
    csv_writer.writerow(["time"]+listx)
    c=0
    for i in data.t.values:
        listinx=[]
        for j in data.d.sel(t=i).values:
            listinx+=[j]
        csv_writer.writerow([i]+listinx)
        c+=1

In [ ]:
###M

variablename="M"
with open(path+"/"+containerid+"_"+variablename+".csv", mode='w',newline='') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"')
    listx=[]
    for x in data.x.values:
        listx+=[x]
    csv_writer.writerow(["time"]+listx)
    c=0
    for i in data.t.values:
        listinx=[]
        for j in data.M.sel(t=i).values:
            listinx+=[j]
        csv_writer.writerow([i]+listinx)
        c+=1

In [ ]:
###S

variablename="S"
with open(path+"/"+containerid+"_"+variablename+".csv", mode='w',newline='') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"')
    listx=[]
    for x in data.x.values:
        listx+=[x]
    csv_writer.writerow(["time"]+listx)
    c=0
    for i in data.t.values:
        listinx=[]
        for j in data.S.sel(t=i).values:
            listinx+=[j]
        csv_writer.writerow([i]+listinx)
        c+=1